# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 04_websearch_context_demo.ipynb

This notebook demonstrates how to use **DropInKnowledgeRuntime** with:

- session-based chat,
- optional RAG (attachments ingested into a vector store),
- **live web search** via `WebSearchExecutor`,

to achieve a "ChatGPT-like" experience with browsing.

The core configuration (LLM adapter, embeddings, vector store, runtime config)
is initialized in a single cell, while the rest of the notebook focuses on
testing and inspecting the web search behavior.


In [ ]:
# 1. Imports and environment setup
# --------------------------------

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import os
from dotenv import load_dotenv

# LLM adapter (Ollama backend via LangChain)
from langchain_ollama import ChatOllama
from intergrax.llm_adapters import LangChainOllamaAdapter

# Embeddings + vector store
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager

# Drop-in knowledge runtime pieces
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig

# Web search integration
from intergrax.websearch.service.websearch_executor import WebSearchExecutor
from intergrax.websearch.providers.google_cse_provider import GoogleCSEProvider
# from intergrax.websearch.providers.bing_provider import BingWebProvider  # optional


# 1.1 Load environment variables (API keys, etc.)
load_dotenv()

# Make sure the env vars for web search are available.
# If you want to use Bing as well, uncomment the Bing provider later.
os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY") or ""
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX") or ""
os.environ["BING_SEARCH_V7_API_KEY"] = os.getenv("BING_SEARCH_V7_API_KEY") or ""


# Core runtime configuration (LLM + embeddings + vector store + web search)

In [ ]:
# 2. Core runtime configuration (LLM + embeddings + vector store + web search)
# -----------------------------------------------------------------------------

# 2.1 Session store – simple in-memory storage for chat messages & metadata.
from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

# 2.2 LLM adapter – here we use Ollama through the LangChain adapter.
llm_adapter = LangChainOllamaAdapter(
    chat=ChatOllama(model=GLOBAL_SETTINGS.default_ollama_model)
)

# 2.3 Embedding manager – MUST match the model used during ingestion.
embedding_manager = EmbeddingManager(
    provider="ollama",
    model_name=GLOBAL_SETTINGS.default_ollama_embed_model,
    assume_ollama_dim=1536,
)

# 2.4 Vector store configuration – same collection as in the RAG demo.
vectorstore_cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs_drop_in_demo",
)

vectorstore_manager = VectorstoreManager(
    config=vectorstore_cfg,
    verbose=True,
)

# 2.5 Web search executor – wraps one or more providers (Google CSE, Bing, etc.)
websearch_executor = WebSearchExecutor(
    providers=[
        GoogleCSEProvider(),
    ],
    max_text_chars=None,
)

# 2.6 RuntimeConfig – single source of truth for drop-in knowledge runtime.
config = RuntimeConfig(
    # LLM & embeddings & vector store
    llm_adapter=llm_adapter,
    embedding_manager=embedding_manager,
    vectorstore_manager=vectorstore_manager,

    # RAG settings (enabled to allow attachment-based context, as in 03 demo)
    enable_rag=True,

    # Web search settings – THIS is the feature under test in this notebook.
    enable_websearch=True,
    websearch_executor=websearch_executor,
    max_docs_per_query=4,  # how many docs we inject into the system prompt

    # Tools / memory – kept off here to isolate the web search behavior.
    enable_tools=False,
    enable_user_profile_memory=True,

    # Optional tenant / workspace (can be overridden per request)
    tenant_id="demo-tenant",
    workspace_id="demo-workspace",
)

# 2.7 Drop-in knowledge runtime – chat engine with RAG + web search.
runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
)

runtime


[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_docs_drop_in_demo
[intergraxVectorstoreManager] Existing count: 100


### 3. Create a fresh chat session for this web search demo

We create a brand new session in the in-memory store.
All questions in this notebook will be sent under the same `session_id`
so that the runtime can keep short-term chat history (like ChatGPT).

Helper: `ask(question: str)` for interactive testing

This helper:
1. Builds a `RuntimeRequest` bound to the current session.
2. Sends it through `DropInKnowledgeRuntime.ask(...)`.
3. Prints:
   - The user question
   - The model answer (truncated optionally)
   - Routing info (was RAG used? was web search used?)
   - Basic debug info (web search, RAG).

In [ ]:
import json
import uuid

from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeAnswer, RuntimeRequest

# 3.1 Create a fresh session for this demo.
session = await session_manager.create_session(
    session_id=str(uuid.uuid4()),
    user_id="demo-user-websearch",
    tenant_id="demo-tenant",
    workspace_id="demo-workspace",
    metadata={"notebook": "04_websearch_drop_in_web_demo"},
)

print("Session created:")
print(f"- id: {session.id}")
print(f"- user_id: {session.user_id}")
print(f"- tenant_id: {session.tenant_id}")
print(f"- workspace_id: {session.workspace_id}")


async def ask(question: str, *, show_full_answer: bool = True) -> RuntimeAnswer:
    """
    Convenience helper for this notebook.

    - Sends the user's question through DropInKnowledgeRuntime.
    - Prints the answer and basic routing/debug information.
    """
    print("\n" + "=" * 80)
    print(f"USER QUESTION:\n{question}")
    print("=" * 80)

    request = RuntimeRequest(
        session_id=session.id,
        user_id=session.user_id,
        tenant_id=session.tenant_id,
        workspace_id=session.workspace_id,
        message=question,
        attachments=[],          # In this notebook we focus on web search (no files)
        metadata={"source": "04_websearch_demo"},
    )

    answer = await runtime.run(request)

    # 1) Print assistant answer
    print("\nASSISTANT ANSWER:\n")
    if show_full_answer:
        print(answer.answer)
    else:
        print(answer.answer[:700] + ("..." if len(answer.answer) > 700 else ""))

    # 2) Routing information (did RAG / websearch fire?)
    print("\n--- ROUTING INFO ---")
    print(f"strategy:           {answer.route.strategy}")
    print(f"used_rag:           {answer.route.used_rag}")
    print(f"used_websearch:     {answer.route.used_websearch}")
    print(f"used_tools:         {answer.route.used_tools}")
    print(f"used_user_profile:  {answer.route.used_user_profile}")

    # 3) Debug trace: show only relevant slices (websearch / rag)
    debug = answer.debug_trace or {}

    print("\n--- DEBUG: RAG ---")
    rag_debug = debug.get("rag")
    if rag_debug:
        print(json.dumps(rag_debug, indent=2, ensure_ascii=False))
    else:
        print("(no RAG debug info)")

    print("\n--- DEBUG: WEBSEARCH ---")
    web_debug = debug.get("websearch")
    if web_debug:
        print(json.dumps(web_debug, indent=2, ensure_ascii=False))
    else:
        print("(no websearch debug info)")

    return answer


answer_2 = await ask(
    "Based on that, which framework would you recommend for a production-ready multi-agent system and why?"
)

answer_2

Session created:
- id: b24d1076-f4a7-4827-9042-ef7acd99bd7a
- user_id: demo-user-websearch
- tenant_id: demo-tenant
- workspace_id: demo-workspace

USER QUESTION:
Based on that, which framework would you recommend for a production-ready multi-agent system and why?

ASSISTANT ANSWER:

The Skreaver framework in Rust (MIT licensed) developed by Oleksandr Husiev seems to be a suitable choice for building production-ready multi-agent systems. It is described as an "agent orchestration framework" that allows for the design and architecture of complex AI systems, making it well-suited for this type of project.

--- ROUTING INFO ---
strategy:           llm_with_websearch
used_rag:           False
used_websearch:     True
used_tools:         False
used_user_profile:  True

--- DEBUG: RAG ---
{
  "enabled": true,
  "used": false,
  "hits_count": 0,
  "where_filter": {
    "session_id": "b24d1076-f4a7-4827-9042-ef7acd99bd7a",
    "user_id": "demo-user-websearch",
    "tenant_id": "demo-tenant",
 

RuntimeAnswer(answer='The Skreaver framework in Rust (MIT licensed) developed by Oleksandr Husiev seems to be a suitable choice for building production-ready multi-agent systems. It is described as an "agent orchestration framework" that allows for the design and architecture of complex AI systems, making it well-suited for this type of project.', citations=[], route=RouteInfo(used_rag=False, used_websearch=True, used_tools=False, used_long_term_memory=False, used_user_profile=True, strategy='llm_with_websearch', extra={}), tool_calls=[], stats=RuntimeStats(total_tokens=None, input_tokens=None, output_tokens=None, rag_tokens=None, websearch_tokens=None, tool_tokens=None, duration_ms=None, extra={}), raw_model_output=None, debug_trace={'session_id': 'b24d1076-f4a7-4827-9042-ef7acd99bd7a', 'user_id': 'demo-user-websearch', 'config': {'llm_label': 'llm-adapter-ollama', 'embedding_label': 'ollama-gte-qwen2-1.5b', 'vectorstore_label': 'chroma-intergrax-docs'}, 'rag': {'enabled': True, 'used